In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install Autogluon
!pip install rdkit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.0/224.0 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 285.7/285.7 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 372.3/372.3 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.8/135.8 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.5/58.5 MB 30.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 93.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.4/90.4 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 5.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 9.5 MB/s eta 0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.7/29.7 MB 45.7 MB/s eta 0:00:00


In [1]:
import pandas as pd
import random
import os
import numpy as np
from sklearn.metrics import mean_squared_error
from tqdm import tqdm
import math
import os
import torch
import random

from autogluon.tabular import TabularDataset, TabularPredictor

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_validate, train_test_split
import itertools
import math
import warnings
warnings.filterwarnings(action='ignore')

c:\Users\ineeji\anaconda3\envs\Ag\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
import pandas as pd
import numpy as np
from sklearn import datasets, ensemble
from catboost import CatBoostRegressor
from tqdm import tqdm
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

from sklearn.model_selection import KFold
from sklearn.metrics import make_scorer, mean_squared_error
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt
import seaborn as sns

from rdkit import Chem
from rdkit.Chem import AllChem, PandasTools
from rdkit import DataStructs

import warnings
warnings.filterwarnings(action='ignore')

In [2]:
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem.Draw import IPythonConsole
from rdkit.Chem import Draw
from rdkit.Chem.Draw.MolDrawing import MolDrawing, DrawingOptions
from rdkit.Chem import Descriptors
DrawingOptions.bondLineWidth=1.8
from rdkit.Chem.rdmolops import SanitizeFlags

# https://github.com/jensengroup/xyz2mol
# from xyz2mol import xyz2mol
from pathlib import Path
import pickle

import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

In [4]:
from datetime import datetime
import pandas as pd
import numpy as np
import datetime as dt
from tqdm.auto import tqdm
# from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
import warnings
import gc
from rdkit import Chem
from rdkit.Chem import AllChem
warnings.filterwarnings("ignore", category=DeprecationWarning)
path = '/content/drive/MyDrive/DACON/bio/new/'

train = pd.read_csv('train_clean.csv')
test = pd.read_csv('test_clean.csv')
submission = pd.read_csv('sample_submission.csv')

In [5]:
duplicates = train[train.duplicated(subset='SMILES', keep=False)]
smiles_duplicate = duplicates['SMILES'].unique()

for i in smiles_duplicate:
    train.reset_index(drop=True, inplace=True)
    df = train[train['SMILES']==i]
    index_list = df.index
    mlm_mean, hlm_mean = np.mean(df['MLM']), np.mean(df['HLM'])
    train = train.drop(index_list[1:])
    train.loc[index_list[0], 'MLM'] = mlm_mean
    train.loc[index_list[0], 'HLM'] = hlm_mean

train.reset_index(drop=True, inplace=True)
test.reset_index(drop=True, inplace=True)

# AllChem

In [6]:
def generate_fingerprint(smiles_string):
    mol = Chem.MolFromSmiles(smiles_string)
    if mol is not None:
        fingerprint = AllChem.GetMorganFingerprintAsBitVect(mol, 2, nBits=2048)
        return list(fingerprint)
    else:
        return [0]*2048

def mol2fp(mol):
    fp = AllChem.GetHashedMorganFingerprint(mol, 6, nBits=4096)
    ar = np.zeros((1,), dtype=np.int8)
    DataStructs.ConvertToNumpyArray(fp, ar)
    return ar

def number_of_atoms(atom_list, df):
    for i in atom_list:
        df['num_of_{}_atoms'.format(i)] = df['mol'].apply(lambda x: len(x.GetSubstructMatches(Chem.MolFromSmiles(i))))

def data_processing(df):
    df_fingerprint = df['SMILES'].apply(generate_fingerprint)
    df_fingerprint = pd.DataFrame(df_fingerprint.tolist(), columns=[f'bit_{i}' for i in range(2048)])
    df = pd.concat([df.drop(columns=['id']), df_fingerprint], axis=1)

    PandasTools.AddMoleculeColumnToFrame(df,'SMILES','Molecule')
    df["FPs"] = df.Molecule.apply(mol2fp)

    df['mol'] = df['SMILES'].apply(lambda x: Chem.MolFromSmiles(x))
    df['mol'] = df['mol'].apply(lambda x: Chem.AddHs(x))
    df['num_of_atoms'] = df['mol'].apply(lambda x: x.GetNumAtoms())
    df['num_of_heavy_atoms'] = df['mol'].apply(lambda x: x.GetNumHeavyAtoms())
    df['tpsa'] = df['mol'].apply(lambda x: Descriptors.TPSA(x))
    df['mol_w'] = df['mol'].apply(lambda x: Descriptors.ExactMolWt(x))
    df['num_valence_electrons'] = df['mol'].apply(lambda x: Descriptors.NumValenceElectrons(x))
    df['num_heteroatoms'] = df['mol'].apply(lambda x: Descriptors.NumHeteroatoms(x))

    number_of_atoms(['C','O', 'N', 'Cl'], df)
    return df

In [14]:
from rdkit.Chem import PandasTools
from rdkit import DataStructs


In [15]:
train = data_processing(train)
test = data_processing(test)

In [16]:
train_mlm = train.drop(columns=['HLM'])
train_hlm = train.drop(columns=['MLM'])

In [17]:
object_columns = train_mlm.select_dtypes(include=['object']).columns
train_mlm = train_mlm.drop(columns=object_columns)

object_columns = train_hlm.select_dtypes(include=['object']).columns
train_hlm = train_hlm.drop(columns=object_columns)

In [21]:
train_data = TabularDataset(train_mlm)
test_data = TabularDataset(test)


predictor = TabularPredictor(label='MLM',  eval_metric='rmse').fit(train_mlm, presets='best_quality',  ag_args_fit={'num_gpus': 0})
predictor.leaderboard(train_mlm, silent=True)


y_pred = predictor.predict(test_data)
y_pred = pd.DataFrame(y_pred, columns=['MLM'])

submission['MLM'] = y_pred['MLM']

No path specified. Models will be saved in: "AutogluonModels\ag-20230913_040522\"
Presets specified: ['best_quality']


Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=8, num_bag_sets=1
	Consider setting `time_limit` to ensure training finishes within an expected duration or experiment with a small portion of `train_data` to identify an ideal `presets` and `hyperparameters` configuration.
Beginning AutoGluon training ...
AutoGluon will save models to "AutogluonModels\ag-20230913_040522\"
AutoGluon Version:  0.8.2
Python Version:     3.10.12
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.22621
Disk Space Avail:   42.47 GB / 511.09 GB (8.3%)
Train Data Rows:    3471
Train Data Columns: 3314
Label Column: MLM
Preprocessing data ...
AutoGluon infers your prediction problem is: 'regression' (because dtype of label-column == float and many unique label-values observed).
	Label info (max, min, mean, stddev): (131.72, 0.0, 37.39137, 35.65765)
	If 'regression' is not the correct problem_type, please manually specify the problem_type parameter during predic

In [ ]:
predictor.leaderboard(train_mlm, silent=True)

In [ ]:
train_data = TabularDataset(train_hlm)
test_data = TabularDataset(test)


predictor = TabularPredictor(label='HLM',  eval_metric='rmse').fit(train_hlm, presets='best_quality',  ag_args_fit={'num_gpus': 0})
predictor.leaderboard(train_hlm, silent=True)


y_pred = predictor.predict(test_data)
y_pred = pd.DataFrame(y_pred, columns=['HLM'])

submission['HLM'] = y_pred['HLM']

No path specified. Models will be saved in: "AutogluonModels/ag-20230912_181511/"
Presets specified: ['high_quality']
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=8, num_bag_sets=1
	Consider setting `time_limit` to ensure training finishes within an expected duration or experiment with a small portion of `train_data` to identify an ideal `presets` and `hyperparameters` configuration.
Beginning AutoGluon training ...
AutoGluon will save models to "AutogluonModels/ag-20230912_181511/"
AutoGluon Version:  0.8.2
Python Version:     3.10.12
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP Fri Jun 9 10:57:30 UTC 2023
Disk Space Avail:   143.37 GB / 179.07 GB (80.1%)
Train Data Rows:    3471
Train Data Columns: 3316
Label Column: HLM
Preprocessing data ...
AutoGluon infers your prediction problem is: 'regression' (because dtype of label-column == float and many unique label-values observed).
	Label info (max, min, mean, stddev): (135.

In [ ]:
predictor.leaderboard(train_hlm, silent=True)

In [ ]:
submission.to_csv('best_qual.csv', index = False, encoding = "utf-8-sig")